## Model Building

#### Image Preprocessing

In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EjE0mWDQeXauvakOV0tuPx7Xadc5zt1Xskl70w1l5tEN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'realtimecommunicationsystempowere-donotdelete-pr-nspaxzxnwuizzf'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/training_set', target_size=(
    64, 64), batch_size=300, class_mode='categorical', color_mode='grayscale')

Found 15750 images belonging to 9 classes.


In [6]:
x_test = test_datagen.flow_from_directory('/home/wsuser/work/test_set', target_size=(
    64, 64), batch_size=300, class_mode='categorical', color_mode='grayscale')

Found 2250 images belonging to 9 classes.


#### Import Libraries

In [7]:
from keras.models import Sequential
from keras.layers import Dropout, Conv2D, MaxPooling2D, Flatten, Dense

#### Model Initialization

In [8]:
model = Sequential()

#### Adding Convolution Layer

In [9]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))

#### Adding Pooling Layer

In [10]:
model.add(MaxPooling2D(pool_size=(2, 2)))

#### Adding Flatten Layer

In [11]:
model.add(Flatten())

#### Adding Dense Layer

In [12]:
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=9, activation='softmax'))

#### Model Compilation

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

#### Fit the Model

In [14]:
model.fit(x_train, steps_per_epoch=24, epochs=10,
          validation_data=x_test, validation_steps=40)

Epoch 1/10
24/24 [==============================] - 32s 1s/step - loss: 1.4646 - accuracy: 0.6543 - val_loss: 0.4951 - val_accuracy: 0.8862
Epoch 2/10
24/24 [==============================] - 30s 1s/step - loss: 0.3022 - accuracy: 0.9204
Epoch 3/10
24/24 [==============================] - 29s 1s/step - loss: 0.1363 - accuracy: 0.9642
Epoch 4/10
24/24 [==============================] - 30s 1s/step - loss: 0.0880 - accuracy: 0.9800
Epoch 5/10
24/24 [==============================] - 29s 1s/step - loss: 0.0609 - accuracy: 0.9851
Epoch 6/10
24/24 [==============================] - 29s 1s/step - loss: 0.0421 - accuracy: 0.9903
Epoch 7/10
24/24 [==============================] - 30s 1s/step - loss: 0.0272 - accuracy: 0.9944
Epoch 8/10
24/24 [==============================] - 28s 1s/step - loss: 0.0228 - accuracy: 0.9946
Epoch 9/10
24/24 [==============================] - 29s 1s/step - loss: 0.0181 - accuracy: 0.9966
Epoch 10/10
24/24 [==============================] - 29s 1s/step - loss: 0.0

#### Save the Model

In [16]:
model.save('model.h5')

In [17]:
!tar -zcvf model.tgz model.h5

model.h5


In [18]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 9.0 MB/s eta 0:00:01


In [19]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
 "url": "https://us-south.ml.cloud.ibm.com",
 "apikey": "L-4G56OudzZ-NVY02d44QriNMoyvZ4UXdVZiOHe2c43j"
}
client = APIClient(wml_credentials)

In [20]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [21]:
space_uid = guid_from_space_name(client, 'Model')
print("Space UID : ", space_uid)

Space UID :  ba7ec6b2-9f4f-4217-b5a5-60cef06d626b


In [22]:
client.set.default_space(space_uid)

'SUCCESS'

In [23]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [24]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [25]:
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [26]:
model_details = client.repository.store_model(model='model.tgz',
meta_props={
 client.repository.ModelMetaNames.NAME: "CNN",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
 client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_id(model_details)

In [27]:
model_id

'a825b731-ca87-4c2b-8491-cc6b6f5d5ef4'

In [28]:
client.repository.download(model_id,'model.tar.gz')

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'

In [29]:
ls

model.h5  model.tar.gz  model.tgz  test_set/  training_set/
